<a href="https://colab.research.google.com/github/sterlinggutterman/Final-Project/blob/main/Final_Merged_Dataset_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Final Merged Dataset Code***

## Set up

In [1]:
! git clone https://github.com/sterlinggutterman/Final-Project

Cloning into 'Final-Project'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 35 (delta 8), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (35/35), 16.14 MiB | 5.26 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# importing data
aisles_df = pd.read_csv('/content/Final-Project/aisles.csv',low_memory=False)
departments_df = pd.read_csv('/content/Final-Project/departments.csv',low_memory=False)
products_df = pd.read_csv('/content/Final-Project/products.csv',low_memory=False)
orders_df = pd.read_csv('/content/orders.csv',low_memory=False)
order_products_prior_df = pd.read_csv('/content/order_products__prior.csv',low_memory=False)

## Products

In [3]:
# merging products datasets
complete_products_df = pd.merge(products_df, aisles_df, on='aisle_id', how='inner')
complete_products_df = pd.merge(complete_products_df, departments_df, on='department_id', how='inner')
complete_products_df.head()

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,2,All-Seasons Salt,104,13,spices seasonings,pantry
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals,frozen
4,5,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry


## Orders

In [4]:
# mergering orders datasets
complete_orders_df = pd.merge(orders_df, order_products_prior_df, on='order_id', how='inner')
complete_orders_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [5]:
# adding a column indicating Yes/No reordered
complete_orders_df['reordered_yes/no'] = np.where(complete_orders_df['reordered'] == 0, ' no', 'yes')

In [7]:
# adding a column indicating the day of the week ordered
complete_orders_df['order_day'] = complete_orders_df['order_dow'].map({0: 'saturday', 1: 'sunday', 2: 'monday',
3: 'tuesday', 4:'wednesday', 5: 'thursday', 6: 'friday'})

In [30]:
# total orders per person
order_sort = complete_orders_df.groupby('user_id')['order_number'].max().sort_values(ascending=False)
order_sort = order_sort.head(2900)

In [31]:
order_sort_merged = pd.merge(complete_orders_df, order_sort, on='user_id', how='inner')
order_sort_merged.head()

,order_id,user_id,eval_set,order_number_x,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,reordered_yes/no,order_day,order_number_y
0,361493,27,prior,1,3,9,NaN,25718,1,0,no,tuesday,81
1,1662354,27,prior,2,2,17,6.0,30776,1,0,no,monday,81
2,1662354,27,prior,2,2,17,6.0,9604,2,0,no,monday,81
3,1662354,27,prior,2,2,17,6.0,6287,3,0,no,monday,81
4,1662354,27,prior,2,2,17,6.0,20947,4,0,no,monday,81


In [32]:
order_sort_merged = order_sort_merged.rename(columns={'order_number_y': 'total_order_count'})

In [33]:
order_sort_merged = order_sort_merged.rename(columns={'order_number_x' : 'order_number'})

In [13]:
order_sort_merged['user_id'].nunique()

3600

In [14]:
order_sort_merged.shape

(2935105, 13)

## Merging products and orders

In [34]:
# mergering orders with products
full_df = pd.merge(order_sort_merged, complete_products_df, on='product_id', how='inner')
full_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,reordered_yes/no,order_day,total_order_count,product_name,aisle_id,department_id,aisle,department
0,361493,27,prior,1,3,9,NaN,25718,1,0,no,tuesday,81,Cold Brew Coffee,26,7,coffee,beverages
1,1662354,27,prior,2,2,17,6.0,30776,1,0,no,monday,81,Organic Raw Kombucha Gingerade,31,7,refrigerated,beverages
2,1662354,27,prior,2,2,17,6.0,9604,2,0,no,monday,81,Hibiscus Organic Raw Kombucha,31,7,refrigerated,beverages
3,1662354,27,prior,2,2,17,6.0,6287,3,0,no,monday,81,Organic & Raw Strawberry Serenity Kombucha,31,7,refrigerated,beverages
4,1662354,27,prior,2,2,17,6.0,20947,4,0,no,monday,81,Passionberry Bliss Kombucha Drink,31,7,refrigerated,beverages


In [27]:
full_df.shape

(2492060, 18)

In [28]:
full_df.sort_values(by='total_order_count', ascending=False)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,reordered_yes/no,order_day,total_order_count,product_name,aisle_id,department_id,aisle,department
1246030,2894357,100982,prior,39,0,18,1.0,43210,2,1,yes,saturday,99,Rolled Oats,68,10,bulk grains rice dried goods,bulk
1353063,3117522,110479,prior,9,1,12,4.0,31801,18,0,no,sunday,99,9 Inch Plates,111,17,plates bowls cups flatware,household
1352941,315081,110479,prior,3,3,14,4.0,13870,2,1,yes,tuesday,99,Lightly Salted Baked Snap Pea Crisps,107,19,chips pretzels,snacks
1352940,315081,110479,prior,3,3,14,4.0,30391,1,1,yes,tuesday,99,Organic Cucumber,83,4,fresh vegetables,produce
1352939,1112707,110479,prior,2,6,14,3.0,19251,8,0,no,friday,99,Almond Smokehouse Nut & Rice Cracker Snacks,78,19,crackers,snacks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886233,1980045,72577,prior,71,0,22,2.0,24852,1,1,yes,saturday,78,Banana,24,4,fresh fruits,produce
886234,1980045,72577,prior,71,0,22,2.0,10863,2,1,yes,saturday,78,Grain Free Turkey & Salmon Formula Cat Food,41,8,cat food care,pets
886235,1980045,72577,prior,71,0,22,2.0,8277,3,1,yes,saturday,78,Apple Honeycrisp Organic,24,4,fresh fruits,produce
886236,1980045,72577,prior,71,0,22,2.0,13968,4,1,yes,saturday,78,Rustic Tomato Basil Sauce,9,9,pasta sauce,dry goods pasta


In [35]:
full_df.to_parquet('full_df.parquet')